# CNN实现MNIST

In [8]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.utils.data as Data
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt5

In [9]:
# Hyper Parameters
BATCH_SIZE = 64
LR = 0.001
EPOCH = 1
DOWNLOAD_MNIST = False

In [10]:
if not(os.path.exists('./mnist')) or not os.listdir('./mnist'):
    DOWNLOAD_MNIST = True

In [11]:
# Training Dataset
train_data = torchvision.datasets.MNIST(
    root = './mnist',
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = DOWNLOAD_MNIST
)

In [12]:
# plot one example
print(train_data.data.size())
print(train_data.targets.size())
plt.imshow(train_data.data[113], cmap='gray')
plt.title('%i'%train_data.targets[113])
plt.show()

torch.Size([60000, 28, 28])
torch.Size([60000])


In [13]:
# Dataloader
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
# Test dataset
test_data = torchvision.datasets.MNIST(root='./mnist', train=False)
print(test_data.data.size())
print(test_data.targets.size())
# pick 2000 samples to testing
# shape(10000, 28, 28) to (10000, 1, 28, 28), value in range(0, 1)  # add a batchsize dimension
test_x = torch.unsqueeze(test_data.data, dim=1).type(torch.FloatTensor)[:2000]/255  
test_y = test_data.targets[:2000]
print(test_x.size())
print(test_y.size())

torch.Size([10000, 28, 28])
torch.Size([10000])
torch.Size([2000, 1, 28, 28])
torch.Size([2000])


In [15]:
# CNN network
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1= nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.output = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # flatten the output of conv2 to (batchsize, 32*7*7)
        output = self.output(x)
        return output, x  # return x for visualization

In [16]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (output): Linear(in_features=1568, out_features=10, bias=True)
)


In [17]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [18]:
# following function (plot_with_labels) is for visualization
from matplotlib import cm
try: from sklearn.manifold import TSNE; HAS_SK = True
except: HAS_SK = False; print('Please install sklearn for layer visualization')
def plot_with_labels(lowDWeights, labels):
    plt.cla()
    X, Y = lowDWeights[:, 0], lowDWeights[:, 1]
    for x, y, s in zip(X, Y, labels):
        c = cm.rainbow(int(255 * s / 9)); plt.text(x, y, s, backgroundcolor=c, fontsize=9)
    plt.xlim(X.min(), X.max()); plt.ylim(Y.min(), Y.max()); plt.title('Visualize last layer'); plt.show(); plt.pause(0.01)


In [19]:
plt.ion()
# training and testing
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):   # gives batch data, normalize x when iterate train_loader

        output = cnn(b_x)[0]               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 50 == 0:
            test_output, last_layer = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
            if HAS_SK:
                # Visualization of trained flatten layer (T-SNE)
                tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
                plot_only = 500
                low_dim_embs = tsne.fit_transform(last_layer.data.numpy()[:plot_only, :])
                labels = test_y.numpy()[:plot_only]
                plot_with_labels(low_dim_embs, labels)
plt.ioff()

Epoch:  0 | train loss: 2.3018 | test accuracy: 0.17
Epoch:  0 | train loss: 0.4713 | test accuracy: 0.85
Epoch:  0 | train loss: 0.2266 | test accuracy: 0.89
Epoch:  0 | train loss: 0.1731 | test accuracy: 0.92
Epoch:  0 | train loss: 0.1893 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1007 | test accuracy: 0.95
Epoch:  0 | train loss: 0.0561 | test accuracy: 0.96
Epoch:  0 | train loss: 0.1849 | test accuracy: 0.96
Epoch:  0 | train loss: 0.1087 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0470 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0639 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0659 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1494 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0549 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0843 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0396 | test accuracy: 0.98
Epoch:  0 | train loss: 0.1163 | test accuracy: 0.98
Epoch:  0 | train loss: 0.0376 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0129 | test accuracy

In [28]:
test_output, _ = cnn(test_x[:10])
# test_output: 网络输出(10, 10)
# torch.max(test_output, 1): 每行的最大值
# torch.max(test_output, 1)[1]： 每行最大值的下标
# torch.max(test_output, 1)[1].data.numpy()：每行最大值
pred_y = torch.max(test_output, 1)[1].data.numpy()  
print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real number')

[7 2 1 0 4 1 4 9 5 9] prediction number
[7 2 1 0 4 1 4 9 5 9] real number
